In [1]:
import papermill as pm

test_cases = (1, 2, 3, 4, 5) # number of days for view, Deschutes data, 1 min interpolation

version = 20
timeout_mins = 4
nb_template = './pm_results/dv_group_days%d_v%d.ipynb'
for days in test_cases: 
    result_nb = nb_template % (days, version)
    print('### Current output NB:', result_nb)
    pm.execute_notebook('./Dataviews_Grouping-PM.ipynb', # input notebook 
                        result_nb,                       # output notebook
                        parameters=dict(
                            dv_days=days,            # number of days for data view 
                            dv_req_timeout=240000,   # request timeout in milliseconds
                            dv_end_num_ferm=6,       # from 1 fermentor up to this number 
                            version=version))

### Current output NB: ./pm_results/dv_group_days1_v20.ipynb



### Current output NB: ./pm_results/dv_group_days2_v20.ipynb



### Current output NB: ./pm_results/dv_group_days3_v20.ipynb



### Current output NB: ./pm_results/dv_group_days4_v20.ipynb



### Current output NB: ./pm_results/dv_group_days5_v20.ipynb


In [2]:
import plotly.graph_objs as go

def generate_trace(df, timeout_x, timeout_y):
    num_ferms, dv_time, dv_status = [], [], []
    for row in df[df.name.str[:4] == 'ferm'].iterrows():
        i, j = row
        num_ferms.append(j.value['num_fermentors'])
        dv_time.append(j.value['total_time'])
        dv_status.append(j.value['status'])
        # print(j.value['num_fermentors'], j.value['total_time'], j.value['status'])
        if 408 == j.value['status']:
            timeout_x.append(j.value['num_fermentors'])
            timeout_y.append(j.value['total_time'])
    trace = go.Scatter(
        x = num_ferms,
        y = dv_time,
        mode = 'lines+markers',
        name = '#days = %d' % days
    )
    return trace, timeout_x, timeout_y 

In [3]:
data, timeout_x, timeout_y = [], [], []
for days in test_cases: 
    nb = pm.read_notebook(nb_template % (days, version))  # extract dataframe results from output NB
    df = nb.dataframe
    trace, timeout_x, timeout_y = generate_trace(df, timeout_x, timeout_y)
    data.append(trace)
# 408 (timeout) 
trace_408 = go.Scatter(
    name='timeout (408)',
    x = timeout_x, 
    y = timeout_y, 
    mode='markers', 
    marker = dict(size=20, color='rgba(152, 0, 0, .5)')
)                
data.append(trace_408)
layout = dict(
    title='Dataviews preview time for Brewing (interpolation=1min, 17-column result CSV)',
    xaxis=dict(title='Number of Fermentors in view', tickvals=[i for i in range(1, 7)]),
    yaxis=dict(title='Time (seconds)'),
)
fig = go.FigureWidget(data=data, layout=layout)
fig

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': '#days = 1',
              'type':…